In [2]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array

In [3]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [4]:
graphPath = "/data/graphDFSample"

<p>1. Create a new column “friend” by exploding of column “friends” (like in the demo iPython notebook)<br>
2. group the resulting dataframe by the column “friend” (like in the demo iPython notebook)<br>
3. create a column “users” by collecting all users with the same id in the column “friend” together (like in the demo iPython notebook)</p>

In [21]:
reversedGraph = sparkSession.read.parquet(graphPath) \
    .withColumn("friend", explode('friends')) \
    .groupBy("friend") \
    .agg(collect_list("user").alias("users")) \
    .withColumn("users_size", size("users"))

reversedGraph.show(3)

+------+--------------------+----------+
|friend|               users|users_size|
+------+--------------------+----------+
|   148|[65051219, 146311...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
+------+--------------------+----------+
only showing top 3 rows



<p>4. sort the elements in the column “users” by the function sort_array</p>

In [22]:
reversedGraph = reversedGraph.select(reversedGraph.friend, \
                                     sort_array(reversedGraph.users).alias("users_sorted"), \
                                     "users_size")
reversedGraph.show(3)

+------+--------------------+----------+
|friend|        users_sorted|users_size|
+------+--------------------+----------+
|   148|[3195315, 1463110...|         4|
|  5518|          [58573511]|         1|
|  9900|          [36844066]|         1|
+------+--------------------+----------+
only showing top 3 rows



<p>5. filter only the rows which have more than 1 element in the column “users”</p>

In [24]:
reversedGraph = reversedGraph.where(reversedGraph.users_size > '1')
reversedGraph.show(3)

+------+--------------------+----------+
|friend|        users_sorted|users_size|
+------+--------------------+----------+
|   148|[3195315, 1463110...|         4|
| 36538|[5506394, 6170161...|        32|
| 41751|[41811068, 60873111]|         2|
+------+--------------------+----------+
only showing top 3 rows



<p>6. for each row emit all possible ordered pairs of users from the column “users”<br>
(tip: write a user defined function for this)</p>

In [ ]:
def user_pairs(users):
    for element in users:
        
    user_agent = re.sub("/?[\d_.]+", "", user_agent)
    user_agent = re.sub("[;\(\):,]", "", user_agent)
    user_agent = user_agent.lower()
    user_agent = user_agent.split()
return user_agent
parse_user_agent_udf = f.udf(parse_user_agent, t.ArrayType(t.StringType()))